In [1]:
import os
import string
import random
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt

In [2]:
# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
#from sklearn.metrics import f1_score

In [3]:
import numpy as np

### Test LABELS for TOKENS in TEST Dataset against BERT Outputs

BERT Classifier has returned results for the tokens passed in 'test.tsv' file.  The returned values are probabilities, that need to be converted into equivalent class labels based on majority class.  Then, the class label should be compared against the actual label from the code above to extract the IO-Coding from the xml files.  This is a brute-force approach or a manual way of verifying the validity of the predictions


Read in results from BERT Predicitons to the above dataset
The above dataset is derived from IO-Coding applied as done on the training set. This is what should be based on the annotation process. Nowe, we have to read in the predictions from bert, which is a set of class probabilities across all 3 classes and we have to merget that with the above dataset for comparison and error analysis.

### Data File Names

* Test files with Labels and Filenames : /data_for_bert_sent/test_files_with_labels/*_testfile.csv
* Bert label mapping /data_for_bert_sent/test_files_with_labels/*_labelmapping.csv
* BERT evaluation /data_for_bert_sent/BERT_run_results/*_eval_results.txt


In [4]:
print(os.path.dirname(os.path.abspath('__file__')))

C:\Users\Kalyan\Documents\Anu\W266 - NLP\Final Project\lheart-disease-risk-prediction\Code


In [5]:
# read in the test files with labels

DI_test = pd.read_csv("data_for_bert_sent/test_files_with_labels/diabetes_ind_testfile.csv")

In [6]:
DI_test.head(10)

,Unnamed: 0,sentence,label,file
0,0,Record date: 2080-02-18,Other,110-03.xml
1,1,SDU JAR Admission Note,Other,110-03.xml
2,2,Name: \t Yosef Villegas,Other,110-03.xml
3,3,MR:\t8249813,Other,110-03.xml
4,4,DOA: \t2/17/80,Other,110-03.xml
5,5,PCP: Gilbert Perez,Other,110-03.xml
6,6,Attending: YBARRA,Other,110-03.xml
7,7,CODE: FULL,Other,110-03.xml
8,8,HPI: 70 yo M with NIDDM admitted for cath aft...,mention,110-03.xml
9,9,Pt has had increasing CP and SOB on exertion f...,Other,110-03.xml


In [7]:
# read in the test results captured for BERT Hypertension model and specify columns as the actual file has no header
bert_DI_results = pd.read_csv("data_for_bert_sent/BERT_run_results/sentence_based_diabetes_ind_results.tsv", sep='\t',header=None)
 
bert_DI_results.columns=["Class0", "Class1", "Class2", "Class3"]

In [8]:
bert_DI_results.head()

,Class0,Class1,Class2,Class3
0,0.000008,0.999957,0.000016,0.000020
1,0.000047,0.998924,0.000069,0.000960
2,0.000030,0.999731,0.000033,0.000207
3,0.000023,0.999789,0.000030,0.000157
4,0.000020,0.999833,0.000027,0.000120


In [9]:
bert_DI_results['predClass'] = bert_DI_results.idxmax(axis=1)

In [10]:
bert_DI_results.head()

,Class0,Class1,Class2,Class3,predClass
0,0.000008,0.999957,0.000016,0.000020,Class1
1,0.000047,0.998924,0.000069,0.000960,Class1
2,0.000030,0.999731,0.000033,0.000207,Class1
3,0.000023,0.999789,0.000030,0.000157,Class1
4,0.000020,0.999833,0.000027,0.000120,Class1


In [11]:
bert_DI_results['predClass'].value_counts()

Class1    34626
Class3      846
Class0       84
Name: predClass, dtype: int64

In [12]:
def DI_set_labels(classlabel):
    if (classlabel=='Class0'):
        return 'A1C'
    elif (classlabel=='Class2'):
        return 'glucose'
    elif (classlabel=='Class3'):
        return 'mention'
    else:
        return 'Other'

In [13]:
bert_DI_results['predLabel'] = bert_DI_results['predClass'].apply(DI_set_labels)
# bert_DI_results.drop('classLabel', 1, inplace=True)

bert_DI_results.head(10)


,Class0,Class1,Class2,Class3,predClass,predLabel
0,0.000008,0.999957,0.000016,0.000020,Class1,Other
1,0.000047,0.998924,0.000069,0.000960,Class1,Other
2,0.000030,0.999731,0.000033,0.000207,Class1,Other
3,0.000023,0.999789,0.000030,0.000157,Class1,Other
4,0.000020,0.999833,0.000027,0.000120,Class1,Other
5,0.000008,0.999956,0.000016,0.000019,Class1,Other
6,0.000008,0.999957,0.000015,0.000020,Class1,Other
7,0.000007,0.999955,0.000014,0.000023,Class1,Other
8,0.000599,0.005017,0.000355,0.994029,Class3,mention
9,0.000010,0.999951,0.000021,0.000018,Class1,Other


In [14]:
# validating the counts by label
bert_DI_results['predLabel'].value_counts()

Other      34626
mention      846
A1C           84
Name: predLabel, dtype: int64

In [15]:
DI_combined = pd.concat([DI_test, bert_DI_results], axis=1)

In [16]:
DI_combined.head()

,Unnamed: 0,sentence,label,file,Class0,Class1,Class2,Class3,predClass,predLabel
0,0,Record date: 2080-02-18,Other,110-03.xml,0.000008,0.999957,0.000016,0.000020,Class1,Other
1,1,SDU JAR Admission Note,Other,110-03.xml,0.000047,0.998924,0.000069,0.000960,Class1,Other
2,2,Name: \t Yosef Villegas,Other,110-03.xml,0.000030,0.999731,0.000033,0.000207,Class1,Other
3,3,MR:\t8249813,Other,110-03.xml,0.000023,0.999789,0.000030,0.000157,Class1,Other
4,4,DOA: \t2/17/80,Other,110-03.xml,0.000020,0.999833,0.000027,0.000120,Class1,Other


In [17]:
DI_combined[DI_combined['predLabel']!='Other']

,Unnamed: 0,sentence,label,file,Class0,Class1,Class2,Class3,predClass,predLabel
8,8,HPI: 70 yo M with NIDDM admitted for cath aft...,mention,110-03.xml,0.000599,0.005017,0.000355,0.994029,Class3,mention
19,19,Diabetes mellitus type 2,mention,110-03.xml,0.000503,0.008200,0.000377,0.990920,Class3,mention
75,75,DM,mention,110-03.xml,0.000553,0.006471,0.000435,0.992542,Class3,mention
105,105,Diabetes mellitus type 2,mention,110-04.xml,0.000503,0.008200,0.000377,0.990920,Class3,mention
183,183,"dependent diabetes mellitus, hypertension.",mention,111-01.xml,0.000509,0.007388,0.000372,0.991731,Class3,mention
220,220,patient is a 58-year-old male with a history o...,Other,111-02.xml,0.000532,0.026905,0.000399,0.972164,Class3,mention
221,221,diabetes who presents with fever and chills.,mention,111-02.xml,0.000496,0.009003,0.000384,0.990117,Class3,mention
291,291,He is 61 years old with previous history of di...,mention,111-03.xml,0.000608,0.006114,0.000357,0.992920,Class3,mention
345,345,"PAST MEDICAL HISTORY: Hypertension, diabetes,...",mention,111-04.xml,0.000517,0.006478,0.000368,0.992638,Class3,mention
426,426,"In addition, CAD, diabetes,",mention,113-04.xml,0.000501,0.008170,0.000378,0.990951,Class3,mention


In [18]:
DI_test_labels = DI_combined['label']
DI_pred_labels = DI_combined['predLabel']

print(type(DI_test_labels))

<class 'pandas.core.series.Series'>


In [19]:
accuracy_score(DI_test_labels, DI_pred_labels)

0.9921813477331534

In [20]:
print(classification_report(DI_pred_labels, DI_test_labels))

C:\Users\Kalyan\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

         A1C       0.66      0.71      0.69        84
       Other       1.00      1.00      1.00     34626
     glucose       0.00      0.00      0.00         0
     mention       0.92      0.85      0.88       846

   micro avg       0.99      0.99      0.99     35556
   macro avg       0.64      0.64      0.64     35556
weighted avg       0.99      0.99      0.99     35556



In [21]:
print("Confusion Matrix \n", confusion_matrix(DI_test_labels, DI_pred_labels))

Confusion Matrix 
 [[   60    20     0    11]
 [   20 34503     0   119]
 [    0    43     0     1]
 [    4    60     0   715]]


In [22]:
unique_label = np.unique(DI_test_labels)
print(pd.DataFrame(confusion_matrix(DI_test_labels, DI_pred_labels, labels=unique_label), 
                   index=['true:{:}'.format(x) for x in unique_label], 
                   columns=['pred:{:}'.format(x) for x in unique_label]))

              pred:A1C  pred:Other  pred:glucose  pred:mention
true:A1C            60          20             0            11
true:Other          20       34503             0           119
true:glucose         0          43             0             1
true:mention         4          60             0           715


In [23]:
DI_combined[(DI_combined['label'] =="Other") & (DI_combined['predLabel'] == "A1C")]

,Unnamed: 0,sentence,label,file,Class0,Class1,Class2,Class3,predClass,predLabel
1601,1601,(8/8/87) A1C 6.10 = 117.,Other,131-01.xml,0.702082,0.267294,0.027608,0.003016,Class0,A1C
1606,1606,1/90 A1C 6.60.,Other,131-01.xml,0.568917,0.408417,0.019275,0.003391,Class0,A1C
1771,1771,(8/8/87) A1C 6.10 = 117.,Other,131-02.xml,0.702082,0.267294,0.027608,0.003016,Class0,A1C
1779,1779,1/90 A1C 6.60.,Other,131-02.xml,0.568917,0.408417,0.019275,0.003391,Class0,A1C
1789,1789,11/92 A1c 7.60 and glyburide added.,Other,131-02.xml,0.982151,0.005687,0.010278,0.001884,Class0,A1C
2539,2539,HBA1C 03/14/2091 6.50,Other,134-03.xml,0.984096,0.003177,0.010233,0.002494,Class0,A1C
3589,3589,Diabetes appears to be in better control accor...,Other,137-03.xml,0.538835,0.038627,0.018296,0.404242,Class0,A1C
12396,12396,He states that his glucose is usually between ...,Other,209-03.xml,0.794944,0.151745,0.049425,0.003886,Class0,A1C
12716,12716,Excellent reduction of hemoglobin A1c to 6%.,Other,210-02.xml,0.981814,0.006484,0.009761,0.001942,Class0,A1C
13373,13373,Last HGBA1C in 11/85 was 7.8.,Other,214-01.xml,0.984508,0.002901,0.010421,0.002170,Class0,A1C


In [24]:
DI_combined[(DI_combined['label'] =="mention") & (DI_combined['predLabel'] == "Other")]

,Unnamed: 0,sentence,label,file,Class0,Class1,Class2,Class3,predClass,predLabel
772,772,Adjust Diabeta if necessary.,mention,115-01.xml,0.000078,0.993868,0.000178,0.005876,Class1,Other
1211,1211,"(2)\tDiabetes mellitus: FBS 126, 137, 149, 12...",mention,117-01.xml,0.000150,0.996142,0.000131,0.003577,Class1,Other
1227,1227,A/P:\t(1)\tDiabetes mellitus: Irregular contro...,mention,117-01.xml,0.000045,0.999568,0.000050,0.000336,Class1,Other
3611,3611,Dr Gil RNHHe has a long history of cardiac dis...,mention,137-04.xml,0.001586,0.949679,0.002169,0.046565,Class1,Other
4688,4688,"HPI: \t69 yo F with PMH of DM, HTN, s/p MI one...",mention,165-04.xml,0.000023,0.999761,0.000029,0.000187,Class1,Other
4825,4825,Phenytoin - TDM PEND 5.0-20.0 mcg/...,mention,165-04.xml,0.000009,0.999949,0.000020,0.000022,Class1,Other
5225,5225,NIGHTFLOAT ADMIT NOTE,mention,167-04.xml,0.000306,0.992263,0.000315,0.007116,Class1,Other
5776,5776,-\tCV/DM diet,mention,168-03.xml,0.000033,0.999618,0.000037,0.000312,Class1,Other
5807,5807,"1.\tdiabetes retinopathy, neuropathy,nephropat...",mention,168-04.xml,0.000078,0.998978,0.000069,0.000876,Class1,Other
5969,5969,NEUROLOGY JUNIOR ADMIT/CONSULT NOTE,mention,169-03.xml,0.000531,0.981517,0.000524,0.017427,Class1,Other


In [ ]:
import numpy as np
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

    fig.tight_layout()
    return ax

plot_confusion_matrix(DI_test_labels, DI_pred_labels, ["AIC", "Other", "glucose", "mention"])
plt.show()

In [ ]:
# read in the test files with labels

DT_test = pd.read_csv("data_for_bert_sent/test_files_with_labels/diabetes_time_testfile.csv")

In [ ]:
DT_test.head(10)

In [ ]:
# read in the test results captured for BERT Hypertension model and specify columns as the actual file has no header
bert_DT_results = pd.read_csv("data_for_bert_sent/BERT_run_results/diabetes_time_test_results.tsv", sep='\t',header=None)
 
bert_DT_results.columns=["Class0", "Class1", "Class2", "Class3"]

In [ ]:
bert_DT_results.head()

In [ ]:
bert_DT_results['predClass'] = bert_DT_results.idxmax(axis=1)

In [ ]:
bert_DT_results.head()

In [ ]:
bert_DT_results['predClass'].value_counts()

In [ ]:
def DT_set_labels(classlabel):
    if (classlabel=='Class1'):
        return 'after DCT'
    elif (classlabel=='Class2'):
        return 'before DCT'
    elif (classlabel=='Class3'):
        return 'during DCT'
    else:
        return 'Other'

In [ ]:
bert_DT_results['predLabel'] = bert_DT_results['predClass'].apply(DT_set_labels)

bert_DT_results.head(10)


In [ ]:
# validating the counts by label
bert_DT_results['predLabel'].value_counts()

In [ ]:
DT_combined = pd.concat([DT_test, bert_DT_results], axis=1)

In [ ]:
DT_combined.head()

In [ ]:
DT_combined[DT_combined['predLabel']!='Other']

In [ ]:
DT_test_labels = DT_combined['label']
DT_pred_labels = DT_combined['predLabel']

#print(type(DT_test_labels))

In [ ]:
accuracy_score(DT_test_labels, DT_pred_labels)

In [ ]:
print(classification_report(DT_pred_labels, DT_test_labels))

In [ ]:
print("Confusion Matrix \n", confusion_matrix(DT_test_labels, DT_pred_labels))

In [ ]:
unique_label = np.unique(DT_test_labels)
print(pd.DataFrame(confusion_matrix(DT_test_labels, DT_pred_labels, labels=unique_label), 
                   index=['true:{:}'.format(x) for x in unique_label], 
                   columns=['pred:{:}'.format(x) for x in unique_label]))

### Interpreting the predictions compared against actual test labels

As seen above, BERT predictions seem very accurate and it seems to predict only after it has seen the complete context.  Also, punctuation marks are not labeled as one of the relevant classes, although a human annotator has done based on the instructions provided as part of the annotation process.  